In [7]:
import pandas as pd
import numpy as np
from collections import defaultdict

In [2]:
q = pd.read_csv('/Users/shanewilliams/Quennect/src/q_data_cleaned2.csv')
f1sub = pd.read_csv('/Users/shanewilliams/Quennect/feb1substations.csv')

In [5]:
print(q.info)
print(q["poi_name"].nunique())

<bound method DataFrame.info of        Unnamed: 0      q_date  q_year  prop_year state entity utility  \
0               0    1/7/2022  2022.0     2024.0    MO    AEC     AEC   
1               3  11/19/2018  2018.0     2021.0    MO    AEC     AEC   
2               6  10/12/2017  2017.0     2020.0    MO    AEC     AEC   
3               8   5/31/2018  2018.0     2020.0    OK    AEC     AEC   
4               9   12/6/2018  2018.0     2021.0    MO    AEC     AEC   
...           ...         ...     ...        ...   ...    ...     ...   
11822       28086   5/31/2022  2022.0     2026.0    CO   TSGT    TSGT   
11823       28099   5/30/2022  2022.0     2025.0    CO   TSGT    TSGT   
11824       28100   5/31/2022  2022.0     2025.0    CO   TSGT    TSGT   
11825       28107   5/31/2022  2022.0     2025.0    CO   TSGT    TSGT   
11826       28124    5/9/2022  2022.0     2025.0    CO   TSGT    TSGT   

         county_1                    poi_name  region_CAISO  ...  type1_Steam  \
0      new

In [6]:
print(f1sub.info)
print(f1sub['name'].nunique())

<bound method DataFrame.info of         OID_  Join_Count  TARGET_FID  Join_Count_1  TARGET_FID_1  OBJECTID  \
0          1           1           9             1             9         9   
1          2           1          11             1            11        11   
2          3           1          12             1            12        12   
3          4           1          13             1            13        13   
4          5           1          14             1            14        14   
...      ...         ...         ...           ...           ...       ...   
58305  58306           1       58225             1         58225     58225   
58306  58307           1       58227             1         58227     58227   
58307  58308           1       58235             1         58235     58235   
58308  58309           1       58251             1         58251     58251   
58309  58310           1       58261             1         58261     58261   

       Join_Count_12  TARGET_FI

In [10]:
f1sub['tokens'] = f1sub['name'].str.lower().str.split()
q['tokens'] = q['poi_name'].str.lower().str.split()

def calc_similarity(tokens1, tokens2):
    set1 = set(tokens1)
    set2 = set(tokens2)
    common_tokens = set1.intersection(set2)
    total_tokens = set1.union(set2)
    return len(common_tokens) / len(total_tokens) if total_tokens else 0

best_match = defaultdict(lambda: {'name': None, 'score': 0})
for _, row1 in f1sub.iterrows():
    for _, row2 in q.iterrows():
        score = calc_similarity(row1['tokens'], row2['tokens'])
        if score > best_match[row1['name']]['score']:
            best_match[row1['name']] = {'name': row2['poi_name'], 'score': score}

matches_df = pd.DataFrame([{'name': k, 'matched_poi_name': v['name'], 'similarity_score': v['score']} for k, v in best_match.items()])

f1sub_matched = f1sub.merge(matches_df, on='name')
final_df = f1sub_matched.merge(q, left_on='matched_poi_name', right_on='poi_name', how='inner')

print(final_df.head())

    OID_  Join_Count  TARGET_FID  Join_Count_1  TARGET_FID_1  OBJECTID  \
0      1           1           9             1             9         9   
1   1567           1        5315             1          5315      5315   
2  15174           1           7             1             7         7   
3  18210           1        5311             1          5311      5311   
4  18211           1        5312             1          5312      5312   

   Join_Count_12  TARGET_FID_12  Join_Count_12_13  TARGET_FID_12_13  ...  \
0              1              9                 1                 9  ...   
1              1           5315                 1              5315  ...   
2              0              7                 1                 7  ...   
3              1           5311                 1              5311  ...   
4              1           5312                 1              5312  ...   

  type1_Waste Heat type1_Wind type1_Wood ia_status_Facility Study  \
0                0          1